In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable. Gemini stuff.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
import random
import nltk
from nltk.corpus import wordnet as wn

nltk.download('wordnet')
noun_synsets = list(wn.all_synsets('n'))
nouns = list(set([noun.name().split('.')[0] for noun in noun_synsets]))

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
def process_model_response(response):
    if "𝕐𝔼𝕊" in response.text:
        return True
    elif "ℕ𝕆" in response.text:
        return False
    else:
        print("The response is unclear. Please try again.")
        return None

In [ ]:
def check_word_relation(word1, word2):
    response = model.generate_content(f"Is {word1} related to {word2} in any subtle, related or obvious manner? If they are related in ANY sort of way then you should say 𝕐𝔼𝕊. Please EXPLAIN your reasoning. If it is related please respond with 𝕐𝔼𝕊, if not ℕ𝕆. Please stick to the blackboard board Unicode characters 𝕐𝔼𝕊/ℕ𝕆 response or else it will not be registered. YOU MUST ALSO EXPLAIN HOW THEY ARE RELATED. For example: Cat is related to fur because cat's have fur so 𝕐𝔼𝕊. YOU MUST USE BLACKBOARD BOLD. YOU SHOULD SAY 𝕐𝔼𝕊 instead of YES.")

    # Display the model's response
    display(to_markdown(response.text))

    return process_model_response(response)


In [ ]:
#Sometimes NLTK words are absolutely confusing.
nltk.download('omw-1.4')
def get_word_definition(word):
    synsets = wn.synsets(word)
    if synsets:
        return synsets[0].definition()
    else:
        return "No definition found."

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [87]:
# Randomly select two nouns
random_nouns = random.sample(nouns, 2)

first_word = random_nouns[0]
second_word = random_nouns[1]

first_word_def = get_word_definition(first_word)
second_word_def = get_word_definition(second_word)

print(f"Your words to connect are: {first_word} and {second_word}")
print(f"Definition of '{first_word}': {first_word_def}")
print(f"Definition of '{second_word}': {second_word_def}")

previous_word = first_word
while True:
    user_input = input(f"Enter a word related to {previous_word}: ")

    # Check relation with the previous word
    is_related_to_previous = check_word_relation(user_input, previous_word)

    if is_related_to_previous is None:
        continue  # Unclear response, ask for input again
    elif not is_related_to_previous:
        print(f"Sorry, '{user_input}' is not related to '{previous_word}'. Try again.")
        continue

    # If related to previous word, now check relation with the second word
    is_related_to_second = check_word_relation(user_input, second_word)

    if is_related_to_second:
        print(f"Congratulations! You have connected '{first_word}' to '{second_word}' through '{user_input}'")
        break
    else:
        print(f"'{user_input}' is related to '{previous_word}' but not to '{second_word}'. Keep going.")
        previous_word = user_input

Your words to connect are: testament and fear
Definition of 'testament': a profession of belief
Definition of 'fear': an emotion experienced in anticipation of some specific pain or danger (usually accompanied by a desire to flee or fight)
Enter a word related to testament: God-fearing


> **𝕐𝔼𝕊.**
> 
> God-fearing is related to testament in the sense that both are associated with religious belief and practice. A testament is a formal declaration of one's beliefs and intentions, often made in a religious context. God-fearing individuals are those who strive to live in accordance with the teachings and principles of their religion. In many religions, testaments are used to express one's faith and commitment to God. Additionally, both testaments and God-fearing behavior are often associated with moral and ethical obligations, such as being truthful, just, and compassionate.

> **𝕐𝔼𝕊.**
> 
> God-fearing is related to fear in the sense that it is a reverence for God that is often accompanied by a sense of awe and respect. This awe and respect can be seen as a form of fear, in the sense that it is a powerful emotion that can elicit feelings of submission and obedience. Additionally, the fear of God can be seen as a motivating factor for righteous behavior, as it encourages individuals to obey God's commands and avoid sin.

Congratulations! You have connected 'testament' to 'fear' through 'God-fearing'
